In [1]:
import cv2
import os
import numpy as np
from tqdm import tqdm
from facenet_pytorch import MTCNN
from PIL import Image
import torch

c:\Users\rajku\anaconda3\envs\new_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def file_path(fake_folder, original_folder):
    fake_videos = [fake_folder+'/'+ f for f in os.listdir(fake_folder) if f.endswith(('.mp4', '.avi'))]
    original_videos = [original_folder+'/'+ f for f in os.listdir(original_folder) if f.endswith(('.mp4', '.avi'))]
    
    video_paths = fake_videos + original_videos
    labels = [1] * len(fake_videos) + [0] * len(original_videos)

    return video_paths, labels

In [3]:
fake_folder = "archive/DFD_manipulated_sequences"
original_folder = "archive/DFD_original sequences"
video_paths, labels = file_path(fake_folder, original_folder)

In [4]:
import os
import cv2
from facenet_pytorch import MTCNN
import numpy as np
from PIL import Image
import torch
from tqdm import tqdm

# Define folder paths
fake_folder = "archive/DFD_manipulated_sequences"
original_folder = "archive/DFD_original sequences"

# Function to gather video paths and labels
def get_video_paths_and_labels(fake_folder, original_folder):
    fake_videos = [os.path.join(fake_folder, f) for f in os.listdir(fake_folder) if f.endswith('.mp4')]
    original_videos = [os.path.join(original_folder, f) for f in os.listdir(original_folder) if f.endswith('.mp4')]
    video_paths = fake_videos + original_videos
    labels = [1] * len(fake_videos) + [0] * len(original_videos)
    return video_paths, labels

# Extract frames with downscaling
def extract_frames(video_path, sample_rate=10, max_size=640):
    vid = cv2.VideoCapture(video_path)
    frames = []
    count = 0
    while vid.isOpened():
        ret, frame = vid.read()
        if not ret:
            break
        if count % sample_rate == 0:
            height, width = frame.shape[:2]
            if max(height, width) > max_size:
                scale = max_size / max(height, width)
                frame = cv2.resize(frame, (int(width * scale), int(height * scale)), interpolation=cv2.INTER_AREA)
            frames.append(frame)
        count += 1
    vid.release()
    return frames

# Batch process with MTCNN on GPU
mtcnn = MTCNN(keep_all=True, image_size=224, margin=20, device='cuda' if torch.cuda.is_available() else 'cpu')

def crop_faces(frames):
    cropped_faces = []
    batch_size = 16  # Process 16 frames at a time
    for i in range(0, len(frames), batch_size):
        batch = frames[i:i + batch_size]
        batch_rgb = [cv2.cvtColor(f, cv2.COLOR_BGR2RGB) for f in batch if f is not None and f.size > 0]
        if not batch_rgb:
            continue
        batch_pil = [Image.fromarray(f) for f in batch_rgb]
        try:
            boxes, _, _ = mtcnn.detect(batch_pil, landmarks=True)
            for j, (frame_rgb, box_set) in enumerate(zip(batch_rgb, boxes)):
                if box_set is not None:
                    
                    for box in box_set:
                        x1, y1, x2, y2 = map(int, box)
                        x1, y1 = max(0, x1), max(0, y1)
                        x2, y2 = min(frame_rgb.shape[1], x2), min(frame_rgb.shape[0], y2)
                        if x2 > x1 and y2 > y1:
                            face = frame_rgb[y1:y2, x1:x2]
                            cropped_faces.append(face)
        except Exception as e:
            print(f"Error processing batch {i//batch_size}: {e}")
    return cropped_faces

# Normalize and resize faces
def normalize_and_resize(faces, size=(224, 224)):
    processed_faces = []
    for face in faces:
        face_img = Image.fromarray(face)
        face_img = face_img.resize(size, Image.Resampling.LANCZOS)
        face_array = np.array(face_img) / 255.0
        processed_faces.append(face_array)
    return processed_faces

# Process a single video
def process_video(video_path, label):
    frames = extract_frames(video_path)

    cropped_faces = crop_faces(frames)
    
    processed_faces = normalize_and_resize(cropped_faces)
    labeled_data = [(face, label) for face in processed_faces]
    return labeled_data

# Test with one video
all_data = []
video_paths, labels = get_video_paths_and_labels(fake_folder, original_folder)
for video_path, label in tqdm(zip(video_paths, labels)):
    labeled_frames = process_video(video_paths[0], labels[0])
    all_data.append(labeled_frames)

2010it [59:55,  1.79s/it]


error: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\core\src\alloc.cpp:73: error: (-4:Insufficient memory) Failed to allocate 6220800 bytes in function 'cv::OutOfMemoryError'


In [8]:
import os
import multiprocessing

print("Logical Cores:", os.cpu_count())  # Includes hyper-threading
print("Physical Cores:", multiprocessing.cpu_count())  # Excludes hyper-threading

Logical Cores: 16
Physical Cores: 16
